### Links
- PyTorch MNIST example - https://github.com/pytorch/examples/blob/master/mnist/main.py
- pytorch cifar10 code - https://github.com/kuangliu/pytorch-cifar/blob/master/main.py
- LeNet for CIFAR-10 - https://github.com/kuangliu/pytorch-cifar/blob/master/models/lenet.py

My Google Doc - https://docs.google.com/document/d/1nPvFULraomLmBp4IuNVe51YXV1DQsSzgQWtuaZhbkjo/edit
Trello - https://docs.google.com/document/d/1nPvFULraomLmBp4IuNVe51YXV1DQsSzgQWtuaZhbkjo/edit
Using webcam - https://docs.google.com/document/d/17BVx5yioNaV11mip4U9gRAO7wLIJf-aOpIITJPjqeks/edit#
Installing pytorch - https://docs.google.com/document/d/1NfLbuhoOxonzS9FEeooyqpB90-62Gat-uZ2DOTaWRdg/edit#


In [1]:
import argparse

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
print(torch.__version__)
print(torch.__file__)

1.0.1.post2
/Users/william/.local/share/virtualenvs/william-oBc2a6gD/lib/python3.7/site-packages/torch/__init__.py


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)  # in_channels, out_channels, kernel_size, stride
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)  # in_features, out_features
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)  # input, kernel_size, stride
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        # reshape for fully connected layers...
        x = x.view(-1, 4*4*50)  # what's the shape after this? 1 x 400?
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x


In [4]:
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')


_StoreAction(option_strings=['--log-interval'], dest='log_interval', nargs=None, const=None, default=10, type=<class 'int'>, choices=None, help='how many batches to wait before logging training status', metavar='N')

In [20]:
args = parser.parse_args("")
args

Namespace(batch_size=64, epochs=10, log_interval=10, lr=0.01, momentum=0.5, no_cuda=False, seed=1, test_batch_size=1000)

In [6]:
use_cuda = not args.no_cuda and torch.cuda.is_available()
use_cuda

False

In [7]:
torch.manual_seed(args.seed)

In [8]:
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cpu')

In [9]:
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
kwargs

{}

In [10]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


In [11]:
model = Net().to(device)
model

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [12]:
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0.5
    nesterov: False
    weight_decay: 0
)

In [13]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # to do: needed?
        optimizer.zero_grad()  # what does this do? zero-out any previous gradient info?
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
            

In [14]:
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300039
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.213460
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.170403
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.076578
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.867874
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.413523
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.000300
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.776155
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.459784
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.485992
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.438883
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.408428
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.460157
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.428866
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.398979
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.385140
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.298582
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.503110
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.523534
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.337511
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.039197
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.150224
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.103794
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.134002
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.140442
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.053784
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.054384
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.028990
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.069645
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.099780
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.028249
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.040082
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.024291
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.040391
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.189066
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.062674
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.044622
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.020471
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.066097
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.085100


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.026283
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.006220
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.060149
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.008353
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.033337
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.015246
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.012088
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.131763
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.061433
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.088719
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.075027
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.029551
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.128522
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.008866
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.032514
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.009720
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.011900
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.054273
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.149792
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.012011


Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.039733
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.014598
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.030364
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.007002
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.011688
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.016178
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.013286
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.013486
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.054743
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.023409
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.018395
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.035109
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.039367
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.001907
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.057063
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.025892
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.003376
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.020058
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.010195
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.023974


Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.011726
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.049374
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.013551
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.033382

Test set: Average loss: 0.0342, Accuracy: 9877/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.004914
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.005661
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.008028
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.014995
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.057178
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.028725
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.024225
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.005671
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.002974
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.010266
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.034286
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.003218
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.009788
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.001021
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.002154
Train Epoch: 8 [9

Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.007738
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.005037
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.025971
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.006101
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.002569
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.033151
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.023379
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.063553
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.047346
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.000988
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.017256
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.001118
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.018744
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.002463
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.009919
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.004928
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.004271
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.034313
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.005449
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.039118


In [15]:
torch.optim.Optimizer

torch.optim.optimizer.Optimizer

In [16]:
type(optimizer)

torch.optim.sgd.SGD

In [17]:
isinstance(optimizer, torch.optim.SGD)

True

In [18]:
issubclass(type(optimizer), torch.optim.Optimizer)

True